

# **Data Science, 2024**



A demonstration of exploratory data analysis to accompany the Lecture.

In [ ]:
import numpy as np
import pandas as pd

These options are used to customize the display of pandas DataFrames for better readability and precision when working with data.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 9)

sns.set()
sns.set_context('talk')

np.set_printoptions(threshold=20, precision=2, suppress=True)
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)
# This option stops scientific notation for pandas
pd.set_option('display.float_format', '{:.2f}'.format)


# Tuberculosis in the United States

What can we say about the presence of Tuberculosis in the United States?

Let's look at the data included in the [original CDC article](https://www.cdc.gov/mmwr/volumes/71/wr/mm7112a1.htm?s_cid=mm7112a1_w#T1_down) published in 2021.

**CSV and Nice Field Names**

Suppose Table 1 was saved as a CSV file located in `data/cdc_tuberculosis.csv`. We can then explore the CSV (which is a text
file, and does not contain binary-encoded data) in many ways:
1. Using a text editor like the one in DataHub (right-click on the file and use `Open->Editor`), emacs, vim, VSCode, etc.
2. Opening the CSV directly in DataHub (read-only), Excel, Google Sheets, etc.
3. The Python file object
4. pandas, using `pd.read_csv()`

Let's start with the first two so we solidify the idea of a CSV as **rectangular data (i.e., tabular data) stored as comma-separated values**.

Try 1, 2.

Then Let's see the tried-and-true Data Science approach: pandas.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Read tuberculosis data
tb = pd.read_csv('/content/drive/My Drive/Data for Pandas/cdc_tuberculosis.csv')
tb

,Unnamed: 0,No. of TB cases,Unnamed: 2,Unnamed: 3,TB incidence,Unnamed: 5,Unnamed: 6
0,U.S. jurisdiction,2019,2020,2021,2019.00,2020.00,2021.00
1,Total,"8,900","7,173","7,860",2.71,2.16,2.37
2,Alabama,87,72,92,1.77,1.43,1.83
3,Alaska,58,58,58,7.91,7.92,7.92
4,Arizona,183,136,129,2.51,1.89,1.77
...,...,...,...,...,...,...,...
48,Virginia,191,169,161,2.23,1.96,1.86
49,Washington,221,163,199,2.90,2.11,2.57
50,West Virginia,9,13,7,0.50,0.73,0.39
51,Wisconsin,51,35,66,0.88,0.59,1.12


Wait, what's up with the "Unnamed" column names? And the first row, for that matter?

Congratulations -- you're ready to wrangle your data. Because of how things are stored, we'll need to clean the data a bit to name our columns better.

A reasonable first step is to identify the row with the right header. The `pd.read_csv()` function ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)) has the convenient `header` parameter:

In [ ]:
# Answer Here
tb = pd.read_csv('/content/drive/My Drive/Data for Pandas/cdc_tuberculosis.csv', header=1)
tb

,U.S. jurisdiction,2019,2020,2021,2019.1,2020.1,2021.1
0,Total,"8,900","7,173","7,860",2.71,2.16,2.37
1,Alabama,87,72,92,1.77,1.43,1.83
2,Alaska,58,58,58,7.91,7.92,7.92
3,Arizona,183,136,129,2.51,1.89,1.77
4,Arkansas,64,59,69,2.12,1.96,2.28
...,...,...,...,...,...,...,...
47,Virginia,191,169,161,2.23,1.96,1.86
48,Washington,221,163,199,2.90,2.11,2.57
49,West Virginia,9,13,7,0.50,0.73,0.39
50,Wisconsin,51,35,66,0.88,0.59,1.12


Wait...but now we can't differentiate betwen the "Number of TB cases" and "TB incidence" year columns. pandas has tried to make our lives easier by automatically adding ".1" to the latter columns, but this doesn't help us as humans understand the data.

We can do this manually with `df.rename()`

In [ ]:
# Rename column List
tb = tb.rename(columns={"2019":"TB cases 2019", "2020":"TB cases 2020","2021":"TB cases 2021","2019.1":"TB incidence 2019","2020.1":"TB incidence 2020","2021.1":"TB incidence 2021",})
tb


,U.S. jurisdiction,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021
0,Total,"8,900","7,173","7,860",2.71,2.16,2.37
1,Alabama,87,72,92,1.77,1.43,1.83
2,Alaska,58,58,58,7.91,7.92,7.92
3,Arizona,183,136,129,2.51,1.89,1.77
4,Arkansas,64,59,69,2.12,1.96,2.28
...,...,...,...,...,...,...,...
47,Virginia,191,169,161,2.23,1.96,1.86
48,Washington,221,163,199,2.90,2.11,2.57
49,West Virginia,9,13,7,0.50,0.73,0.39
50,Wisconsin,51,35,66,0.88,0.59,1.12


# Record Granularity

You might already be wondering: What's up with that first record?

Row 0 is what we call a **rollup record**, or summary record. It's often useful when displaying tables to humans. The **granularity** of record 0 (Totals) vs the rest of the records (States) is different.


Okay, EDA step two. How was the rollup record aggregated?

Let's check if Total TB cases is the sum of all state TB cases. If we sum over all rows, we should get **2x** the total cases in each of our TB cases by year (why?).

In [ ]:
# Apply sum to tb DataFrame
tb.sum()


,0
U.S. jurisdiction,TotalAlabamaAlaskaArizonaArkansasCaliforniaCol...
TB cases 2019,"8,9008758183642,111666718245583029973261085237..."
TB cases 2020,"7,1737258136591,706525417194122219282169239376..."
TB cases 2021,"7,8609258129691,750585443194992281064255127494..."
TB incidence 2019,109.94
TB incidence 2020,93.09
TB incidence 2021,102.94



Whoa, what's going on? Check out the column types:

In [ ]:
# Find datatype for each column
tb.dtypes

,0
U.S. jurisdiction,object
TB cases 2019,object
TB cases 2020,object
TB cases 2021,object
TB incidence 2019,float64
TB incidence 2020,float64
TB incidence 2021,float64


Looks like those commas are causing all TB cases to be read as the `object` datatype, or **storage type** (close to the Python string datatype), so pandas is concatenating strings instead of adding integers.

Fortunately `read_csv` also has a `thousands` parameter (for what it's worth, I didn't know this beforehand--I [googled](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) this):

In [ ]:
tb[['TB cases 2019', 'TB cases 2020', 'TB cases 2021']] = tb[['TB cases 2019', 'TB cases 2020', 'TB cases 2021']].replace(',', '', regex=True).astype(int)
tb


,U.S. jurisdiction,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021
0,Total,8900,7173,7860,2.71,2.16,2.37
1,Alabama,87,72,92,1.77,1.43,1.83
2,Alaska,58,58,58,7.91,7.92,7.92
3,Arizona,183,136,129,2.51,1.89,1.77
4,Arkansas,64,59,69,2.12,1.96,2.28
...,...,...,...,...,...,...,...
47,Virginia,191,169,161,2.23,1.96,1.86
48,Washington,221,163,199,2.90,2.11,2.57
49,West Virginia,9,13,7,0.50,0.73,0.39
50,Wisconsin,51,35,66,0.88,0.59,1.12


In [ ]:
# now apply sum
# Answer Here
tb.sum()

,0
U.S. jurisdiction,TotalAlabamaAlaskaArizonaArkansasCaliforniaCol...
TB cases 2019,17800
TB cases 2020,14346
TB cases 2021,15720
TB incidence 2019,109.94
TB incidence 2020,93.09
TB incidence 2021,102.94


In [ ]:
tb

,U.S. jurisdiction,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021
0,Total,8900,7173,7860,2.71,2.16,2.37
1,Alabama,87,72,92,1.77,1.43,1.83
2,Alaska,58,58,58,7.91,7.92,7.92
3,Arizona,183,136,129,2.51,1.89,1.77
4,Arkansas,64,59,69,2.12,1.96,2.28
...,...,...,...,...,...,...,...
47,Virginia,191,169,161,2.23,1.96,1.86
48,Washington,221,163,199,2.90,2.11,2.57
49,West Virginia,9,13,7,0.50,0.73,0.39
50,Wisconsin,51,35,66,0.88,0.59,1.12


In [ ]:
df_state_level = tb[tb['U.S. jurisdiction'] != 'Total']
df_state_level

,U.S. jurisdiction,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021
1,Alabama,87,72,92,1.77,1.43,1.83
2,Alaska,58,58,58,7.91,7.92,7.92
3,Arizona,183,136,129,2.51,1.89,1.77
4,Arkansas,64,59,69,2.12,1.96,2.28
5,California,2111,1706,1750,5.35,4.32,4.46
...,...,...,...,...,...,...,...
47,Virginia,191,169,161,2.23,1.96,1.86
48,Washington,221,163,199,2.90,2.11,2.57
49,West Virginia,9,13,7,0.50,0.73,0.39
50,Wisconsin,51,35,66,0.88,0.59,1.12


The Total TB cases look right. Phew!

(We'll leave it to your own EDA to figure out how the TB incidence "Totals" were aggregated.)

Let's just look at the records with **state-level granularity**:

In [ ]:
# Answer Here
df_state_level = tb[tb['U.S. jurisdiction'] != 'Total']
df_state_level

,U.S. jurisdiction,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021
1,Alabama,87,72,92,1.77,1.43,1.83
2,Alaska,58,58,58,7.91,7.92,7.92
3,Arizona,183,136,129,2.51,1.89,1.77
4,Arkansas,64,59,69,2.12,1.96,2.28
5,California,2111,1706,1750,5.35,4.32,4.46
...,...,...,...,...,...,...,...
47,Virginia,191,169,161,2.23,1.96,1.86
48,Washington,221,163,199,2.90,2.11,2.57
49,West Virginia,9,13,7,0.50,0.73,0.39
50,Wisconsin,51,35,66,0.88,0.59,1.12


In [ ]:
df_state_level = tb[tb['U.S. jurisdiction'] != 'Total']
df_state_level

,U.S. jurisdiction,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021
1,Alabama,87,72,92,1.77,1.43,1.83
2,Alaska,58,58,58,7.91,7.92,7.92
3,Arizona,183,136,129,2.51,1.89,1.77
4,Arkansas,64,59,69,2.12,1.96,2.28
5,California,2111,1706,1750,5.35,4.32,4.46
...,...,...,...,...,...,...,...
47,Virginia,191,169,161,2.23,1.96,1.86
48,Washington,221,163,199,2.90,2.11,2.57
49,West Virginia,9,13,7,0.50,0.73,0.39
50,Wisconsin,51,35,66,0.88,0.59,1.12


What do each of these values represent? Why?

To the lecture!


# Gather Census Data

U.S. Census population estimates [source](https://www.census.gov/data/tables/time-series/demo/popest/2010s-state-total.html) (2019), [source](https://www.census.gov/data/tables/time-series/demo/popest/2020s-state-total.html) (2020-2021).

Running the below cells cleans the data. We encourage you to closely explore the CSV and study these lines after lecture...

There are a few new methods here:
* `df.convert_dtypes()` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.convert_dtypes.html)) conveniently converts all float dtypes into ints and is out of scope for the class.
* `df.drop_na()` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html)) will be explained in more detail next time.

In [ ]:
# Load 2010s census data
# Answer
tb1 = pd.read_csv('/content/nst-est2019-01.csv')
tb1
tb1 = pd.read_csv('/content/nst-est2019-01.csv', header=3)
tb1

,Unnamed: 0,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,United States,"308,745,538","308,758,105","309,321,666","311,556,874","313,830,990","315,993,715","318,301,008","320,635,163","322,941,311","324,985,539","326,687,501","328,239,523"
1,Northeast,"55,317,240","55,318,443","55,380,134","55,604,223","55,775,216","55,901,806","56,006,011","56,034,684","56,042,330","56,059,240","56,046,620","55,982,803"
2,Midwest,"66,927,001","66,929,725","66,974,416","67,157,800","67,336,743","67,560,379","67,745,167","67,860,583","67,987,540","68,126,781","68,236,628","68,329,004"
3,South,"114,555,744","114,563,030","114,866,680","116,006,522","117,241,208","118,364,400","119,624,037","120,997,341","122,351,760","123,542,189","124,569,433","125,580,448"
4,West,"71,945,553","71,946,907","72,100,436","72,788,329","73,477,823","74,167,130","74,925,793","75,742,555","76,559,681","77,257,329","77,834,820","78,347,268"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Note: The estimates are based on the 2010 Cens...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,Suggested Citation:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,Table 1. Annual Estimates of the Resident Popu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,"Source: U.S. Census Bureau, Population Division",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Apply some EDA

Drop the column of Estiamte Base

Rename Unnamed: 0 as 'Geographic Area'

"smart" converting of columns(.convert_dtypes()) Google it. Use at your own risk

.dropna() to drop records with NaN

You can also suggest any change that can be helpful for EDA



In [ ]:
#Drop the column of Estiamte Base
#tb1.drop('Estimates Base', axis=1, inplace=True)
#Rename Unnamed: 0 as 'Geographic Area'
tb1.rename(columns={'Unnamed: 0' : 'Geographic Area'}, inplace=True)
#"smart" converting of columns(.convert_dtypes()) Google it. Use at your own risk
tb1.convert_dtypes()
#.dropna() to drop records with NaN
tb1.dropna()
#You can also suggest any change that can be helpful for EDA
tb1

,Geographic Area,Census,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,United States,"308,745,538","309,321,666","311,556,874","313,830,990","315,993,715","318,301,008","320,635,163","322,941,311","324,985,539","326,687,501","328,239,523"
1,Northeast,"55,317,240","55,380,134","55,604,223","55,775,216","55,901,806","56,006,011","56,034,684","56,042,330","56,059,240","56,046,620","55,982,803"
2,Midwest,"66,927,001","66,974,416","67,157,800","67,336,743","67,560,379","67,745,167","67,860,583","67,987,540","68,126,781","68,236,628","68,329,004"
3,South,"114,555,744","114,866,680","116,006,522","117,241,208","118,364,400","119,624,037","120,997,341","122,351,760","123,542,189","124,569,433","125,580,448"
4,West,"71,945,553","72,100,436","72,788,329","73,477,823","74,167,130","74,925,793","75,742,555","76,559,681","77,257,329","77,834,820","78,347,268"
...,...,...,...,...,...,...,...,...,...,...,...,...
52,.Washington,"6,724,540","6,742,830","6,826,627","6,897,058","6,963,985","7,054,655","7,163,657","7,294,771","7,423,362","7,523,869","7,614,893"
53,.West Virginia,"1,852,994","1,854,239","1,856,301","1,856,872","1,853,914","1,849,489","1,842,050","1,831,023","1,817,004","1,804,291","1,792,147"
54,.Wisconsin,"5,686,986","5,690,475","5,705,288","5,719,960","5,736,754","5,751,525","5,760,940","5,772,628","5,790,186","5,807,406","5,822,434"
55,.Wyoming,"563,626","564,487","567,299","576,305","582,122","582,531","585,613","584,215","578,931","577,601","578,759"


In [ ]:
# census 2020s data
tb2 = pd.read_csv('/content/NST-EST2022-POP.csv', header=2)
tb2

,Geographic Area,"April 1, 2020 Estimates Base",Population Estimate (as of July 1),Unnamed: 3,Unnamed: 4
0,NaN,NaN,2020,2021,2022
1,United States,"331,449,520","331,511,512","332,031,554","333,287,557"
2,Northeast,"57,609,156","57,448,898","57,259,257","57,040,406"
3,Midwest,"68,985,537","68,961,043","68,836,505","68,787,595"
4,South,"126,266,262","126,450,613","127,346,029","128,716,192"
...,...,...,...,...,...
59,Note: The estimates are developed from a base ...,NaN,NaN,NaN,NaN
60,Suggested Citation:,NaN,NaN,NaN,NaN
61,Annual Estimates of the Resident Population fo...,NaN,NaN,NaN,NaN
62,"Source: U.S. Census Bureau, Population Division",NaN,NaN,NaN,NaN


In [ ]:
# perform EDA Here as did on 2010s census data
tb2.rename(columns={'Population Estimate (as of July 1)':'2020','Unnamed: 3' : '2021','Unnamed: 4' : '2022',}, inplace=True)
#"smart" converting of columns(.convert_dtypes()) Google it. Use at your own risk
tb2.convert_dtypes()
#.dropna() to drop records with NaN
tb2.dropna(inplace=True)
#You can also suggest any change that can be helpful for EDA
tb2

,Geographic Area,"April 1, 2020 Estimates Base",2020,2021,2022
1,United States,"331,449,520","331,511,512","332,031,554","333,287,557"
2,Northeast,"57,609,156","57,448,898","57,259,257","57,040,406"
3,Midwest,"68,985,537","68,961,043","68,836,505","68,787,595"
4,South,"126,266,262","126,450,613","127,346,029","128,716,192"
5,West,"78,588,565","78,650,958","78,589,763","78,743,364"
...,...,...,...,...,...
53,.Washington,"7,705,247","7,724,031","7,740,745","7,785,786"
54,.West Virginia,"1,793,755","1,791,420","1,785,526","1,775,156"
55,.Wisconsin,"5,893,725","5,896,271","5,880,101","5,892,539"
56,.Wyoming,"576,837","577,605","579,483","581,381"


In [84]:
tb1.rename(columns={'Geographic Area' : 'Geographic Area_x',}, inplace=True)
tb2.rename(columns={'Geographic Area_x' : 'Geographic Area_y',}, inplace=True)
c_tb1 = tb1
c_tb2 = tb2
gax = c_tb1.drop(columns=['Census'])
gay = c_tb2.drop(columns=['April 1, 2020 Estimates Base'])
merged = pd.merge(left = gax, right = gay, left_on='Geographic Area_x', right_on='Geographic Area_y')

m_again = pd.merge(left = merged, right = df_state_level, left_on = 'Geographic Area_x', right_on = 'U.S. jurisdiction')
m_again

,Geographic Area_x,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Geographic Area_y,2020,2021,2022,U.S. jurisdiction,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021




# Join Data (Merge DataFrames)


Time to `merge`! Here I use the DataFrame method `df1.merge(right=df2, ...)` on DataFrame `df1` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html)). Contrast this with the function `pd.merge(left=df1, right=df2, ...)` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.merge.html?highlight=pandas%20merge#pandas.merge)). Feel free to use either.

In [ ]:
# merge TB dataframe with two US census dataframes
tb1.rename(columns={'Geographic Area' : 'Geographic Area_x',}, inplace=True)
tb2.rename(columns={'Geographic Area_x' : 'Geographic Area_y',}, inplace=True)
c_tb1 = tb1
c_tb2 = tb2
gax = c_tb1.drop(columns=['Census'])
gay = c_tb2.drop(columns=['April 1, 2020 Estimates Base'])
merged = pd.merge(left = gax, right = gay, left_on='Geographic Area_x', right_on='Geographic Area_y')

m_again = pd.merge(left = merged, right = df_state_level, left_on = 'Geographic Area_x', right_on = 'U.S. jurisdiction')
m_again

,U.S. jurisdiction,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021,Geographic Area_x,2010,2011,...,2014,2015,2016,2017,2018,2019,Geographic Area_y,2020,2021,2022
0,Alabama,87,72,92,1.77,1.43,1.83,Alabama,4785437,4799069,...,4841799,4852347,4863525,4874486,4887681,4903185,Alabama,5031362,5049846,5074296
1,Alaska,58,58,58,7.91,7.92,7.92,Alaska,713910,722128,...,736283,737498,741456,739700,735139,731545,Alaska,732923,734182,733583
2,Arizona,183,136,129,2.51,1.89,1.77,Arizona,6407172,6472643,...,6730413,6829676,6941072,7044008,7158024,7278717,Arizona,7179943,7264877,7359197
3,Arkansas,64,59,69,2.12,1.96,2.28,Arkansas,2921964,2940667,...,2967392,2978048,2989918,3001345,3009733,3017804,Arkansas,3014195,3028122,3045637
4,California,2111,1706,1750,5.35,4.32,4.46,California,37319502,37638369,...,38596972,38918045,39167117,39358497,39461588,39512223,California,39501653,39142991,39029342
5,Colorado,66,52,58,1.15,0.90,1.00,Colorado,5047349,5121108,...,5350101,5450623,5539215,5611885,5691287,5758736,Colorado,5784865,5811297,5839926
6,Connecticut,67,54,54,1.88,1.50,1.50,Connecticut,3579114,3588283,...,3594524,3587122,3578141,3573297,3571520,3565287,Connecticut,3597362,3623355,3626205
7,Delaware,18,17,43,1.84,1.71,4.29,Delaware,899593,907381,...,932487,941252,948921,956823,965479,973764,Delaware,992114,1004807,1018396
8,District of Columbia,24,19,19,3.39,2.75,2.84,District of Columbia,605226,619800,...,662328,675400,685815,694906,701547,705749,District of Columbia,670868,668791,671803
9,Florida,558,412,499,2.60,1.91,2.29,Florida,18845537,19053237,...,19845911,20209042,20613477,20963613,21244317,21477737,Florida,21589602,21828069,22244823


This is a little unwieldy. We could either drop the unneeded columns now, or just merge on smaller census DataFrames. Let's do the latter.

In [ ]:
# try merging again, but cleaner this time




## Reproduce incidence

Let's recompute incidence to make sure we know where the original CDC numbers came from.

From the [CDC report](https://www.cdc.gov/mmwr/volumes/71/wr/mm7112a1.htm?s_cid=mm7112a1_w#T1_down): TB incidence is computed as “Cases per 100,000 persons using mid-year population estimates from the U.S. Census Bureau.”

If we define a group as 100,000 people, then we can compute the TB incidence for a given state population as

$$\text{TB incidence} = \frac{\text{# TB cases in population}}{\text{# groups in population}} = \frac{\text{# TB cases in population}}{\text{population}/100000} $$

$$= \frac{\text{# TB cases in population}}{\text{population}} \times 100000$$

Let's try this for 2019:

In [88]:
tb_census_df = tb
#tb_census_df["recompute incidence 2019"] = tb_census_df["TB cases 2019"]/tb_census_df["2019"]*100000
tb_cases_in_population = tb.loc[1, 'U.S. jusdriction']

population =
groups_in_population = population*100000


SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (<ipython-input-88-43dc368a048a>, line 4)

Awesome!!!

Let's use a for-loop and Python format strings to compute TB incidence for all years. Python f-strings are just used for the purposes of this demo, but they're handy to know when you explore data beyond this course ([Python documentation](https://docs.python.org/3/tutorial/inputoutput.html)).

In [ ]:
# recompute incidence for all years (2019, 2020, 2021)


,U.S. jurisdiction,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021,Geographic Area_x,2010,2011,...,2017,2018,2019,Geographic Area_y,2020,2021,2022,recompute incidence 2019,recompute incidence 2020,recompute incidence 2021
0,Alabama,87,72,92,1.77,1.43,1.83,Alabama,4785437,4799069,...,4874486,4887681,4903185,Alabama,5031362,5049846,5074296,1.774357,1.431024,1.821838
1,Alaska,58,58,58,7.91,7.92,7.92,Alaska,713910,722128,...,739700,735139,731545,Alaska,732923,734182,733583,7.928425,7.913519,7.899949
2,Arizona,183,136,129,2.51,1.89,1.77,Arizona,6407172,6472643,...,7044008,7158024,7278717,Arizona,7179943,7264877,7359197,2.514179,1.894165,1.775667
3,Arkansas,64,59,69,2.12,1.96,2.28,Arkansas,2921964,2940667,...,3001345,3009733,3017804,Arkansas,3014195,3028122,3045637,2.120747,1.957405,2.27864
4,California,2111,1706,1750,5.35,4.32,4.46,California,37319502,37638369,...,39358497,39461588,39512223,California,39501653,39142991,39029342,5.342651,4.318807,4.470788
5,Colorado,66,52,58,1.15,0.90,1.00,Colorado,5047349,5121108,...,5611885,5691287,5758736,Colorado,5784865,5811297,5839926,1.146085,0.898897,0.998056
6,Connecticut,67,54,54,1.88,1.50,1.50,Connecticut,3579114,3588283,...,3573297,3571520,3565287,Connecticut,3597362,3623355,3626205,1.879232,1.5011,1.490331
7,Delaware,18,17,43,1.84,1.71,4.29,Delaware,899593,907381,...,956823,965479,973764,Delaware,992114,1004807,1018396,1.848497,1.713513,4.279429
8,District of Columbia,24,19,19,3.39,2.75,2.84,District of Columbia,605226,619800,...,694906,701547,705749,District of Columbia,670868,668791,671803,3.400642,2.832152,2.840947
9,Florida,558,412,499,2.60,1.91,2.29,Florida,18845537,19053237,...,20963613,21244317,21477737,Florida,21589602,21828069,22244823,2.598039,1.908326,2.286047


These numbers look pretty close!!! There are a few errors in the hundredths place, particularly in 2021. It may be useful to further explore reasons behind this discrepancy. We'll leave it to you!

In [89]:
tb_census_df.describe()

,table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
count,64,58,59,58,58
unique,64,58,59,58,58
top,Annual Estimates of the Resident Population fo...,"April 1, 2020 Estimates Base",Population Estimate (as of July 1),2021,2022
freq,1,1,1,1,1
